## My Twitter Bot

My twitter bot tweets out if the US flag is supposed to be at half staff for either a holiday, or by proclomation. From the website us.halfstaff.org it scrapes for country wide event date for the current or upcoming half-staff notification, and from flagnotify.com it scrapes for upcoming dates for flags at half-staff for particular states. It then tweets out the cominbed information including countrywide and state notifications. 

The scraping for us.halfstaff.org looks at the most recent posting for notifications and grabs the date. For flagnotify.com the bot scrapes the upcoming notifications for the top post for the date and state. Any changes to these and it should update the google sheet and send out a tweet with the new information.

Both websites do not have any terms of use and do not restrict robot scraping.
the robots.txt for halfstaff.org returned the results:
User-agent: *
Disallow: /wp-admin/
Allow: /wp-admin/admin-ajax.php
Flagnotify.com/robots.txt returned the restuls:
User-agent: *
Disallow: /images/
Disallow: /fs/
Disallow: /app_code/
Disallow: /admin/
Disallow: /transfer/
Disallow: /util.aspx
Disallow: /movies

I had many iterations of the bot scraping from different websites, but finally found two with the information I wanted in an available form. At first I scraped two parts of the us.halfstaff.org website for the information date, reason for the half-staff, and by whom it was declared, but without any new updates, the parse code I had for the declaration may not have worked, so instead I switched the second parse to individual states and had to get rid of the reason to shorten the tweet to be able to be sent, but added a link to more information on why it was to be half-staff.


In [161]:
import urllib.request
import re 
import datetime
now = datetime.datetime.now()

In [162]:
url_1 = "http://www.us.halfstaff.org/"
url_2 = "http://flagnotify.com/scheduled.aspx"

In [163]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds']

document_key = "17x26DRzJFIDYJxTnULtV0wUpuWLPrt5Y7W9hnZGZTUY"
credentials = ServiceAccountCredentials.from_json_keyfile_name('../../../../../TweetyBot-b2d02f67ab13.json', scope)

gc = gspread.authorize(credentials)
wks = gc.open_by_key(document_key)

worksheet = wks.worksheet("Sheet1")

worksheet.resize(worksheet.row_count)

In [164]:

import csv
csvfile = "output.csv"
list_of_lists = worksheet.get_all_values()
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerows(list_of_lists)

import pandas as pd
output = pd.read_csv(csvfile)
output[:3]

,timestamp,url_1,url_2,url_3,all together now
0,0,0,0,0,0


In [165]:
import twitter
from twitter import TwitterError

with open('../../../../../key.txt', 'r') as myfile:
    key=myfile.read()
    
with open('../../../../../secret.txt', 'r') as myfile:
    secret=myfile.read()
    
with open('../../../../../token_key.txt', 'r') as myfile:
    token_key=myfile.read()

with open('../../../../../token_secret.txt', 'r') as myfile:
    token_secret=myfile.read()

api = twitter.Api(consumer_key=key,
                  consumer_secret=secret,
                  access_token_key=token_key,
                  access_token_secret=token_secret)

In [166]:
p_1 = urllib.request.build_opener(urllib.request.HTTPCookieProcessor).open(url_1).read()
print(p_1)

b'\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\r\n<html xmlns="http://www.w3.org/1999/xhtml" dir="ltr" lang="en-US">\r\n<head profile="http://gmpg.org/xfn/11">\r\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\r\n<title>UNITED STATES OF AMERICA Half Staff Flag Status</title>\r\n<link rel="stylesheet" href="http://halfstaff.org/wp-content/themes/halfstaff/style.css" type="text/css" media="screen" />\r\n<link rel="alternate" type="application/rss+xml" title="HALF STAFF AMERICAN FLAG NOTIFICATIONS RSS Feed" href="http://halfstaff.org/feed/" />\r\n<link rel="alternate" type="application/atom+xml" title="HALF STAFF AMERICAN FLAG NOTIFICATIONS Atom Feed" href="http://halfstaff.org/feed/atom/" />\r\n<link rel="pingback" href="http://halfstaff.org/xmlrpc.php" />\r\n<style type="text/css" media="screen">\r\n#page {\r\n\tbackground: url("http://halfstaff.org/wp-content/themes/halfstaff/images/

In [167]:
res_1 = re.search(b'\t<div class=\"date\">START: <strong>(.*)</strong></div>',p_1)
print(res_1.group(1).decode('UTF-8'))

10/2/2017


In [168]:
p_2 = urllib.request.build_opener(urllib.request.HTTPCookieProcessor).open(url_2).read()
print(p_2)

b'\r\n\r\n<!DOCTYPE html>\r\n<html>\r\n  <head><meta charset="UTF-8"><title>\r\n\tUpcoming Lowerings - FlagNotify\r\n</title><link href="favicon.ico" rel="shortcut icon" type="image/x-icon" /><meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" name="viewport">\r\n    <!-- Bootstrap 3.3.2 -->\r\n    <link href="bootstrap/css/bootstrap.min.css" rel="stylesheet" type="text/css" />\r\n    <!-- Font Awesome Icons --> \r\n      <link href="https://maxcdn.bootstrapcdn.com/font-awesome/4.3.0/css/font-awesome.min.css" rel="stylesheet" type="text/css" />\r\n     \r\n    <!-- Theme style -->\r\n    <link href="dist/css/AdminLTE.css" rel="stylesheet" type="text/css" />\r\n\r\n\r\n      <!-- REQUIRED JS SCRIPTS -->\r\n      <!-- jQuery 2.1.3 -->\r\n      <script src="libraries/jQuery-2.1.3.min.js"></script>\r\n      <!-- Bootstrap 3.3.2 JS -->\r\n      <script src="bootstrap/js/bootstrap.min.js" type="text/javascript"></script>\r\n\r\n      <script src="dist/js/ap

In [169]:
res_2 = re.search(b'<td  style = \"font-weight:normal;\">(\D*)</td>',p_2)  
print (res_2.group(1).decode('UTF-8'))

New York


In [170]:
p_3 = urllib.request.build_opener(urllib.request.HTTPCookieProcessor).open(url_2).read()
print(p_3)

b'\r\n\r\n<!DOCTYPE html>\r\n<html>\r\n  <head><meta charset="UTF-8"><title>\r\n\tUpcoming Lowerings - FlagNotify\r\n</title><link href="favicon.ico" rel="shortcut icon" type="image/x-icon" /><meta content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no" name="viewport">\r\n    <!-- Bootstrap 3.3.2 -->\r\n    <link href="bootstrap/css/bootstrap.min.css" rel="stylesheet" type="text/css" />\r\n    <!-- Font Awesome Icons --> \r\n      <link href="https://maxcdn.bootstrapcdn.com/font-awesome/4.3.0/css/font-awesome.min.css" rel="stylesheet" type="text/css" />\r\n     \r\n    <!-- Theme style -->\r\n    <link href="dist/css/AdminLTE.css" rel="stylesheet" type="text/css" />\r\n\r\n\r\n      <!-- REQUIRED JS SCRIPTS -->\r\n      <!-- jQuery 2.1.3 -->\r\n      <script src="libraries/jQuery-2.1.3.min.js"></script>\r\n      <!-- Bootstrap 3.3.2 JS -->\r\n      <script src="bootstrap/js/bootstrap.min.js" type="text/javascript"></script>\r\n\r\n      <script src="dist/js/ap

In [171]:
res_3 = re.search(b'weight:bold;\">(.*)</td>',p_3)  
print (res_3.group(1).decode('UTF-8'))

10/25/2017


In [172]:
if res_1 and res_2 and res_3: 
    output_1 = res_1.group(1).decode('UTF-8')
    output_2 = res_2.group(1).decode('UTF-8')
    output_3 = res_3.group(1).decode('UTF-8')
    titles = output_1 + " and in " + output_2 + " on " + output_3 + ". For more information visit https://goo.gl/7fQNMg"
print("%s | %s | %s | %s | %s"%(worksheet.row_values(worksheet.row_count)[1],worksheet.row_values(worksheet.row_count)[2],worksheet.row_values(worksheet.row_count)[3],worksheet.row_values(worksheet.row_count)[4],worksheet.row_values(worksheet.row_count)[2]))

0 | 0 | 0 | 0 | 0


In [173]:
print("%s | %s | %s | %s | %s"%(worksheet.row_values(worksheet.row_count)[1],worksheet.row_values(worksheet.row_count)[2],worksheet.row_values(worksheet.row_count)[3],worksheet.row_values(worksheet.row_count)[4],worksheet.row_values(worksheet.row_count)[2]))

0 | 0 | 0 | 0 | 0


In [174]:
print("%s | %s | %s | %s | %s"%(now,output_1,output_2,output_3,titles))

2017-10-24 23:37:45.602271 | 10/2/2017 | New York | 10/25/2017 | 10/2/2017 and in New York on 10/25/2017. For more information visit https://goo.gl/7fQNMg


In [175]:
if res_1 and res_2: 
 
    
    if (float(worksheet.row_values(worksheet.row_count)[1]) != output_1) or (float(worksheet.row_values(worksheet.row_count)[2]) != output_2):
        
        if (float(worksheet.row_values(worksheet.row_count)[1]) != output_1):
 
            try:
                #print ('Titles: %s'%(output_2))
                status = api.PostUpdate('US Flags are Half-Staff countrywide on %s'%(titles))
                print(status.text)
            except TwitterError:
                #print ('T1 + T2: %s'%(output_2))
                status = api.PostUpdate('US Flags to be Half-Staff countrywide on %s'%(titles))
                print(status.text)
                
        worksheet.append_row([now,output_1,output_2,output_3,titles])

US Flags are Half-Staff countrywide on 10/2/2017 and in New York on 10/25/2017. For more information visit https://t.co/LQlK38yorl
